In [2]:
1

1

In [6]:
!pip install paramiko

     |████████████████████████████████| 225 kB 5.7 MB/s eta 0:00:01
     |████████████████████████████████| 273 kB 43.0 MB/s eta 0:00:01
     |████████████████████████████████| 856 kB 45.6 MB/s eta 0:00:01


In [7]:
import io
import os
import re
import glob
import pandas as pd
import numpy as np

from datetime import datetime
import datetime as dt
import time
import boto3

import zipfile
import pytz

from datetime import date, timedelta
import gzip
import pandas as pd
from pandas.io.json import json_normalize 
import json
import gnupg
from io import StringIO
import boto3
import paramiko
import os
import requests
import logging
import os
import typing
from os.path import join, dirname
import time
import logging
from paramiko import SFTPClient, SFTPFile, Message, SFTPError, Transport
from paramiko.sftp import CMD_STATUS, CMD_READ, CMD_DATA

logger = logging.getLogger('demo')


In [ ]:
class _SFTPFileDownloader:
    """
    Helper class to download large file with paramiko sftp client with limited number of concurrent requests.
    """

    _DOWNLOAD_MAX_REQUESTS = 48
    _DOWNLOAD_MAX_CHUNK_SIZE = 0x8000

    def __init__(self, f_in: SFTPFile, f_out: typing.BinaryIO, callback=None):
        self.f_in = f_in
        self.f_out = f_out
        self.callback = callback

        self.requested_chunks = {}
        self.received_chunks = {}
        self.saved_exception = None

    def download(self):
        file_size = self.f_in.stat().st_size
        requested_size = 0
        received_size = 0

        while True:
            # send read requests
            while len(self.requested_chunks) + len(self.received_chunks) < self._DOWNLOAD_MAX_REQUESTS and \
                    requested_size < file_size:
                chunk_size = min(self._DOWNLOAD_MAX_CHUNK_SIZE, file_size - requested_size)
                request_id = self._sftp_async_read_request(
                    fileobj=self,
                    file_handle=self.f_in.handle,
                    offset=requested_size,
                    size=chunk_size
                )
                self.requested_chunks[request_id] = (requested_size, chunk_size)
                requested_size += chunk_size

            # receive blocks if they are available
            # note: the _async_response is invoked
            self.f_in.sftp._read_response()
            self._check_exception()

            # write received data to output stream
            while True:
                chunk = self.received_chunks.pop(received_size, None)
                if chunk is None:
                    break
                _, chunk_size, chunk_data = chunk
                self.f_out.write(chunk_data)
                if self.callback is not None:
                    self.callback(chunk_data)

                received_size += chunk_size

            # check transfer status
            if received_size >= file_size:
                break

            # check chunks queues
            if not self.requested_chunks and len(self.received_chunks) >= self._DOWNLOAD_MAX_REQUESTS:
                raise ValueError("SFTP communication error. The queue with requested file chunks is empty and"
                                 "the received chunks queue is full and cannot be consumed.")

        return received_size

    def _sftp_async_read_request(self, fileobj, file_handle, offset, size):
        sftp_client = self.f_in.sftp

        with sftp_client._lock:
            num = sftp_client.request_number

            msg = Message()
            msg.add_int(num)
            msg.add_string(file_handle)
            msg.add_int64(offset)
            msg.add_int(size)

            sftp_client._expecting[num] = fileobj
            sftp_client.request_number += 1

        sftp_client._send_packet(CMD_READ, msg)
        return num

    def _async_response(self, t, msg, num):
        if t == CMD_STATUS:
            # save exception and re-raise it on next file operation
            try:
                self.f_in.sftp._convert_status(msg)
            except Exception as e:
                self.saved_exception = e
            return
        if t != CMD_DATA:
            raise SFTPError("Expected data")
        data = msg.get_string()

        chunk_data = self.requested_chunks.pop(num, None)
        if chunk_data is None:
            return

        # save chunk
        offset, size = chunk_data

        if size != len(data):
            raise SFTPError(f"Invalid data block size. Expected {size} bytes, but it has {len(data)} size")
        self.received_chunks[offset] = (offset, size, data)

    def _check_exception(self):
        """if there's a saved exception, raise & clear it"""
        if self.saved_exception is not None:
            x = self.saved_exception
            self.saved_exception = None
            raise x


def download_file(sftp_client: SFTPClient, remote_path: str, local_path: str, callback=None):
    """
    Helper function to download remote file via sftp.
    It contains a fix for a bug that prevents a large file downloading with :meth:`paramiko.SFTPClient.get`
    Note: this function relies on some private paramiko API and has been tested with paramiko 2.7.1.
          So it may not work with other paramiko versions.
    :param sftp_client: paramiko sftp client
    :param remote_path: remote file path
    :param local_path: local file path
    :param callback: optional data callback
    """
    remote_file_size = sftp_client.stat(remote_path).st_size

    with sftp_client.open(remote_path, 'rb') as f_in, open(local_path, 'wb') as f_out:
        _SFTPFileDownloader(
            f_in=f_in,
            f_out=f_out,
            callback=callback
        ).download()

    local_file_size = os.path.getsize(local_path)
    if remote_file_size != local_file_size:
        raise IOError(f"file size mismatch: {remote_file_size} != {local_file_size}")


def main():

    start_time = time.time()
    # public sftp server
    host = 'DOTS.datalabusa.com'
    port = 22
    username = 'ex_Avant'
    password = 'Et#h@ut8Ec2F'
    
    
    ### Getting file list
    
    ssh = paramiko.SSHClient()
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())

    ssh.connect(hostname=host,port=port,username=username,password=password)
    sftp = ssh.open_sftp()

    file_list = sftp.listdir('From_Datalab')
    campaign_month = '202409'
    campaign_files = 'DL_AVT_REMAILED_202409_20240826.csv.pgp'

    ssh.close()
    sftp.close()
    
    
    for file in campaign_files:
    
        remote_file_path = 'From_Datalab/{}'.format(file)
    #     local_file_path = join(dirname(__file__), 'DL_AVT_INTERIMFILE2_202408_HI.csv.pgp')
        local_file_path = '/tmp/{}'.format(file)

        transport = Transport((host, port))
        transport.set_keepalive(30)
        transport.connect(
            username=username,
            password=password,
        )

        with SFTPClient.from_transport(transport) as sftp_client:
            progress_size = 0
            total_size = 0
            step_size = 4 * 1024 * 1024

            def progress_callback(data):
                nonlocal progress_size, total_size
                progress_size += len(data)
                total_size += len(data)
                while progress_size >= step_size:
                    logger.info(f"{total_size // (1024 ** 2)} MB has been downloaded")
                    progress_size -= step_size

            download_file(
                sftp_client=sftp_client,
                remote_path=remote_file_path,
                local_path=local_file_path,
                callback=progress_callback
            )
        download_time = time.time() - start_time
        print(f"File downloaded successfully to {local_file_path} in {download_time:.2f} seconds")

        # s4 upload
        s3_bucket_name = 'avant-partner01-landing'
        s3_object_key = 'datalabs/drop/mailed/{file}'.format(
        campaign = campaign_month,
        file = file
        )
        s3_resource = boto3.resource("s3")
        s3_resource.Bucket(name=s3_bucket_name).upload_file(
        Filename=local_file_path, Key=s3_object_key
        )


        # Clean up local file
        os.remove(local_file_path)
    
if __name__ == '__main__':
    logging.basicConfig(level=logging.INFO, format="%(asctime)s [%(levelname)s]: %(message)s")
    main()



2024-07-28 04:28:18,136 [INFO]: Connected (version 2.0, client MOVEit)
2024-07-28 04:28:18,846 [INFO]: Authentication (password) successful!
2024-07-28 04:28:18,920 [INFO]: [chan 0] Opened sftp connection (server version 3)
2024-07-28 04:28:19,125 [INFO]: [chan 0] sftp session closed.
2024-07-28 04:28:19,178 [INFO]: Connected (version 2.0, client MOVEit)
2024-07-28 04:28:19,809 [INFO]: Authentication (password) successful!
2024-07-28 04:28:19,884 [INFO]: [chan 0] Opened sftp connection (server version 3)
2024-07-28 04:28:20,738 [INFO]: 4 MB has been downloaded
2024-07-28 04:28:21,415 [INFO]: 8 MB has been downloaded
2024-07-28 04:28:21,989 [INFO]: 12 MB has been downloaded
2024-07-28 04:28:22,607 [INFO]: 16 MB has been downloaded
2024-07-28 04:28:23,150 [INFO]: 20 MB has been downloaded
2024-07-28 04:28:23,690 [INFO]: 24 MB has been downloaded
2024-07-28 04:28:24,300 [INFO]: 28 MB has been downloaded
2024-07-28 04:28:24,865 [INFO]: 32 MB has been downloaded
2024-07-28 04:28:25,468 [INF

File downloaded successfully to /tmp/DL_AVT_INTERIMFILE1_202409_1.csv.pgp in 257.29 seconds


2024-07-28 04:32:58,778 [INFO]: Connected (version 2.0, client MOVEit)
2024-07-28 04:32:59,408 [INFO]: Authentication (password) successful!
2024-07-28 04:32:59,485 [INFO]: [chan 0] Opened sftp connection (server version 3)
2024-07-28 04:33:00,318 [INFO]: 4 MB has been downloaded
2024-07-28 04:33:00,899 [INFO]: 8 MB has been downloaded
2024-07-28 04:33:01,553 [INFO]: 12 MB has been downloaded
2024-07-28 04:33:02,177 [INFO]: 16 MB has been downloaded
2024-07-28 04:33:02,780 [INFO]: 20 MB has been downloaded
2024-07-28 04:33:03,416 [INFO]: 24 MB has been downloaded
2024-07-28 04:33:04,009 [INFO]: 28 MB has been downloaded
2024-07-28 04:33:04,603 [INFO]: 32 MB has been downloaded
2024-07-28 04:33:05,217 [INFO]: 36 MB has been downloaded
2024-07-28 04:33:05,841 [INFO]: 40 MB has been downloaded
2024-07-28 04:33:06,427 [INFO]: 44 MB has been downloaded
2024-07-28 04:33:06,991 [INFO]: 48 MB has been downloaded
2024-07-28 04:33:07,612 [INFO]: 52 MB has been downloaded
2024-07-28 04:33:08,180 

File downloaded successfully to /tmp/DL_AVT_INTERIMFILE1_202409_10.csv.pgp in 527.52 seconds


2024-07-28 04:37:27,959 [INFO]: Connected (version 2.0, client MOVEit)
2024-07-28 04:37:28,636 [INFO]: Authentication (password) successful!
2024-07-28 04:37:28,711 [INFO]: [chan 0] Opened sftp connection (server version 3)
2024-07-28 04:37:29,584 [INFO]: 4 MB has been downloaded
2024-07-28 04:37:30,199 [INFO]: 8 MB has been downloaded
2024-07-28 04:37:30,813 [INFO]: 12 MB has been downloaded
2024-07-28 04:37:31,435 [INFO]: 16 MB has been downloaded
2024-07-28 04:37:32,001 [INFO]: 20 MB has been downloaded
2024-07-28 04:37:32,605 [INFO]: 24 MB has been downloaded
2024-07-28 04:37:33,174 [INFO]: 28 MB has been downloaded
2024-07-28 04:37:33,810 [INFO]: 32 MB has been downloaded
2024-07-28 04:37:34,439 [INFO]: 36 MB has been downloaded
2024-07-28 04:37:35,035 [INFO]: 40 MB has been downloaded
2024-07-28 04:37:35,631 [INFO]: 44 MB has been downloaded
2024-07-28 04:37:36,277 [INFO]: 48 MB has been downloaded
2024-07-28 04:37:36,827 [INFO]: 52 MB has been downloaded
2024-07-28 04:37:37,428 

...

2024-07-28 05:34:43,005 [INFO]: Connected (version 2.0, client MOVEit)
2024-07-28 05:34:43,686 [INFO]: Authentication (password) successful!
2024-07-28 05:34:43,756 [INFO]: [chan 0] Opened sftp connection (server version 3)
2024-07-28 05:34:44,541 [INFO]: 4 MB has been downloaded
2024-07-28 05:34:45,133 [INFO]: 8 MB has been downloaded
2024-07-28 05:34:45,778 [INFO]: 12 MB has been downloaded
2024-07-28 05:34:46,324 [INFO]: 16 MB has been downloaded
2024-07-28 05:34:46,933 [INFO]: 20 MB has been downloaded
2024-07-28 05:34:47,537 [INFO]: 24 MB has been downloaded
2024-07-28 05:34:48,123 [INFO]: 28 MB has been downloaded
2024-07-28 05:34:48,667 [INFO]: 32 MB has been downloaded
2024-07-28 05:34:49,174 [INFO]: 36 MB has been downloaded
2024-07-28 05:34:49,767 [INFO]: 40 MB has been downloaded
2024-07-28 05:34:50,371 [INFO]: 44 MB has been downloaded
2024-07-28 05:34:50,935 [INFO]: 48 MB has been downloaded
2024-07-28 05:34:51,522 [INFO]: 52 MB has been downloaded
2024-07-28 05:34:52,067 

File downloaded successfully to /tmp/DL_AVT_INTERIMFILE1_202409_6.csv.pgp in 4227.22 seconds


2024-07-28 05:39:07,836 [INFO]: Connected (version 2.0, client MOVEit)
2024-07-28 05:39:08,437 [INFO]: Authentication (password) successful!
2024-07-28 05:39:08,512 [INFO]: [chan 0] Opened sftp connection (server version 3)
2024-07-28 05:39:09,364 [INFO]: 4 MB has been downloaded
2024-07-28 05:39:09,989 [INFO]: 8 MB has been downloaded
2024-07-28 05:39:10,593 [INFO]: 12 MB has been downloaded
2024-07-28 05:39:11,179 [INFO]: 16 MB has been downloaded
2024-07-28 05:39:11,807 [INFO]: 20 MB has been downloaded
2024-07-28 05:39:12,413 [INFO]: 24 MB has been downloaded
2024-07-28 05:39:13,049 [INFO]: 28 MB has been downloaded
2024-07-28 05:39:13,664 [INFO]: 32 MB has been downloaded
2024-07-28 05:39:14,269 [INFO]: 36 MB has been downloaded
2024-07-28 05:39:14,913 [INFO]: 40 MB has been downloaded
2024-07-28 05:39:15,563 [INFO]: 44 MB has been downloaded
2024-07-28 05:39:16,182 [INFO]: 48 MB has been downloaded
2024-07-28 05:39:16,746 [INFO]: 52 MB has been downloaded
2024-07-28 05:39:17,335 

File downloaded successfully to /tmp/DL_AVT_INTERIMFILE1_202409_7.csv.pgp in 4506.70 seconds


2024-07-28 05:43:47,795 [INFO]: Connected (version 2.0, client MOVEit)
2024-07-28 05:43:48,445 [INFO]: Authentication (password) successful!
2024-07-28 05:43:48,521 [INFO]: [chan 0] Opened sftp connection (server version 3)
2024-07-28 05:43:49,408 [INFO]: 4 MB has been downloaded
2024-07-28 05:43:50,048 [INFO]: 8 MB has been downloaded
2024-07-28 05:43:50,621 [INFO]: 12 MB has been downloaded
2024-07-28 05:43:51,240 [INFO]: 16 MB has been downloaded
2024-07-28 05:43:51,875 [INFO]: 20 MB has been downloaded
2024-07-28 05:43:52,474 [INFO]: 24 MB has been downloaded
2024-07-28 05:43:53,091 [INFO]: 28 MB has been downloaded
2024-07-28 05:43:53,708 [INFO]: 32 MB has been downloaded
2024-07-28 05:43:54,332 [INFO]: 36 MB has been downloaded
2024-07-28 05:43:54,977 [INFO]: 40 MB has been downloaded
2024-07-28 05:43:55,572 [INFO]: 44 MB has been downloaded
2024-07-28 05:43:56,170 [INFO]: 48 MB has been downloaded
2024-07-28 05:43:56,782 [INFO]: 52 MB has been downloaded
2024-07-28 05:43:57,379 

2024-07-28 04:20:51,349 [INFO]: Connected (version 2.0, client MOVEit)
2024-07-28 04:20:51,960 [INFO]: Authentication (password) successful!
2024-07-28 04:20:52,036 [INFO]: [chan 0] Opened sftp connection (server version 3)


2024-07-28 04:21:04,548 [INFO]: [chan 0] sftp session closed.


In [62]:
campaign_files

['DL_AVT_INTERIMFILE1_202409_1.csv.pgp',
 'DL_AVT_INTERIMFILE1_202409_10.csv.pgp',
 'DL_AVT_INTERIMFILE1_202409_11.csv.pgp',
 'DL_AVT_INTERIMFILE1_202409_12.csv.pgp',
 'DL_AVT_INTERIMFILE1_202409_13.csv.pgp',
 'DL_AVT_INTERIMFILE1_202409_14.csv.pgp',
 'DL_AVT_INTERIMFILE1_202409_15.csv.pgp',
 'DL_AVT_INTERIMFILE1_202409_16.csv.pgp',
 'DL_AVT_INTERIMFILE1_202409_17.csv.pgp',
 'DL_AVT_INTERIMFILE1_202409_18.csv.pgp',
 'DL_AVT_INTERIMFILE1_202409_19.csv.pgp',
 'DL_AVT_INTERIMFILE1_202409_2.csv.pgp',
 'DL_AVT_INTERIMFILE1_202409_3.csv.pgp',
 'DL_AVT_INTERIMFILE1_202409_4.csv.pgp',
 'DL_AVT_INTERIMFILE1_202409_5.csv.pgp',
 'DL_AVT_INTERIMFILE1_202409_6.csv.pgp',
 'DL_AVT_INTERIMFILE1_202409_7.csv.pgp',
 'DL_AVT_INTERIMFILE1_202409_8.csv.pgp',
 'DL_AVT_INTERIMFILE1_202409_9.csv.pgp',
 'DL_AVT_INTERIMFILE2_202409_1.csv.pgp',
 'DL_AVT_INTERIMFILE2_202409_10.csv.pgp',
 'DL_AVT_INTERIMFILE2_202409_11.csv.pgp',
 'DL_AVT_INTERIMFILE2_202409_12.csv.pgp',
 'DL_AVT_INTERIMFILE2_202409_13.csv.pgp',
 '

In [12]:
def main():
    start_time = time.time()
    # public sftp server
    host = 'DOTS.datalabusa.com'
    port = 22
    username = 'ex_Avant'
    password = 'Et#h@ut8Ec2F'
    remote_file_path = 'From_Datalab/DL_AVT_INTERIMFILE1_202409_1.csv.pgp'
    #     local_file_path = join(dirname(__file__), 'DL_AVT_INTERIMFILE2_202408_HI.csv.pgp')
    local_file_path = '/tmp/DL_AVT_INTERIMFILE1_202409_1.csv.pgp'

    transport = Transport((host, port))
    transport.set_keepalive(30)
    transport.connect(
        username=username,
        password=password,
    )

    with SFTPClient.from_transport(transport) as sftp_client:
        progress_size = 0
        total_size = 0
        step_size = 4 * 1024 * 1024

        def progress_callback(data):
            nonlocal progress_size, total_size
            progress_size += len(data)
            total_size += len(data)
            while progress_size >= step_size:
                logger.info(f"{total_size // (1024 ** 2)} MB has been downloaded")
                progress_size -= step_size

        download_file(
            sftp_client=sftp_client,
            remote_path=remote_file_path,
            local_path=local_file_path,
            callback=progress_callback
        )
    download_time = time.time() - start_time
    print(f"File downloaded successfully to {local_file_path} in {download_time:.2f} seconds")

In [13]:
main()

2024-07-28 03:49:24,946 [INFO]: Connected (version 2.0, client MOVEit)
2024-07-28 03:49:25,590 [INFO]: Authentication (password) successful!
2024-07-28 03:49:25,660 [INFO]: [chan 0] Opened sftp connection (server version 3)
2024-07-28 03:49:26,609 [INFO]: 4 MB has been downloaded
2024-07-28 03:49:27,216 [INFO]: 8 MB has been downloaded
2024-07-28 03:49:27,823 [INFO]: 12 MB has been downloaded
2024-07-28 03:49:28,408 [INFO]: 16 MB has been downloaded
2024-07-28 03:49:29,032 [INFO]: 20 MB has been downloaded
2024-07-28 03:49:29,637 [INFO]: 24 MB has been downloaded
2024-07-28 03:49:30,262 [INFO]: 28 MB has been downloaded
2024-07-28 03:49:30,867 [INFO]: 32 MB has been downloaded
2024-07-28 03:49:31,491 [INFO]: 36 MB has been downloaded
2024-07-28 03:49:32,095 [INFO]: 40 MB has been downloaded
2024-07-28 03:49:32,626 [INFO]: 44 MB has been downloaded
2024-07-28 03:49:33,223 [INFO]: 48 MB has been downloaded
2024-07-28 03:49:33,828 [INFO]: 52 MB has been downloaded
2024-07-28 03:49:34,474 

File downloaded successfully to /tmp/DL_AVT_INTERIMFILE1_202409_1.csv.pgp in 242.82 seconds


In [14]:
local_file_path

'/tmp/DL_AVT_INTERIMFILE1_202409_1.csv.pgp'

In [35]:
s3_resource.Bucket(name=s3_bucket_name).upload_file(
    Filename=local_file_path, Key=s3_object_key
)

In [10]:
def main():

    start_time = time.time()
    # public sftp server
    host = 'DOTS.datalabusa.com'
    port = 22
    username = 'ex_Avant'
    password = 'Et#h@ut8Ec2F'
    remote_file_path = 'From_Datalab/DL_AVT_INTERIMFILE1_202409_1.csv.pgp'
#     local_file_path = join(dirname(__file__), 'DL_AVT_INTERIMFILE2_202408_HI.csv.pgp')
    local_file_path = '/tmp/DL_AVT_INTERIMFILE1_202409_1.csv.pgp'

    transport = Transport((host, port))
    transport.set_keepalive(30)
    transport.connect(
        username=username,
        password=password,
    )
    
    with SFTPClient.from_transport(transport) as sftp_client:
        progress_size = 0
        total_size = 0
        step_size = 4 * 1024 * 1024

        def progress_callback(data):
            nonlocal progress_size, total_size
            progress_size += len(data)
            total_size += len(data)
            while progress_size >= step_size:
                logger.info(f"{total_size // (1024 ** 2)} MB has been downloaded")
                progress_size -= step_size
        
        download_file(
            sftp_client=sftp_client,
            remote_path=remote_file_path,
            local_path=local_file_path,
            callback=progress_callback
        )
    download_time = time.time() - start_time
    print(f"File downloaded successfully to {local_file_path} in {download_time:.2f} seconds")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
main()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkStatementCancellationFailedException: Interrupted by user but Livy failed to cancel the Spark statement. The Livy session might have become unusable.

In [30]:
# s4 upload
s3_bucket_name = 'avantcamp-partner01-prod-publish-bucket'
s3_object_key = 'avant/direct_mail/first_mail/datalabs_interim_file/test_garden/DL_AVT_INTERIMFILE1_202409_1.csv.pgp'


# Initialize the S3 client
s3_client = boto3.client('s3')

# Step 1: Initiate the multipart upload
response = s3_client.create_multipart_upload(Bucket=s3_bucket_name, Key=s3_object_key)
upload_id = response['UploadId']
print(f'Upload ID: {upload_id}')

# Step 2: Upload parts
part_size = 10 * 1024 * 1024  # 5 MB
part_info = {'Parts': []}
part_number = 1


Upload ID: bsUNFafmN13u5KNAtBVqwud_HyUdb4W4WeBhD.ROITfLusIysIjq6Gk_nyDKGNnx5Lv6UgBjZjSII.wVEdal9JXYIE4G2KQ0uoJDy8x91S3ABKuxc6z23N9VQhOf6Y6_prblbGbdjC9WhvhwT6516g531wjPKN65JAnwkQVLShU-


In [31]:
with open('/tmp/DL_AVT_INTERIMFILE1_202409_1.csv.pgp', 'rb') as file:
    while True:
        data = file.read(part_size)
        if not data:
            break

        # Generate pre-signed URL for the part
        presigned_url = s3_client.generate_presigned_url(
            ClientMethod='upload_part',
            Params={
                'Bucket': s3_bucket_name,
                'Key': s3_object_key,
                'UploadId': upload_id,
                'PartNumber': part_number,
            }
        )


        # Upload the part
        response = requests.put(presigned_url, data=data)
        print(response)
        etag = response.headers['ETag']

        # Keep track of uploaded part details
        part_info['Parts'].append({
            'PartNumber': part_number
        })

        print(f'Uploaded part {part_number} ')
        part_number += 1

# Step 3: Complete the multipart upload
response = s3_client.complete_multipart_upload(
    Bucket=s3_bucket_name,
    Key=s3_object_key,
    UploadId=upload_id,
    MultipartUpload=part_info
)
print(f'Multipart upload completed: {response}')

# Clean up local file
# os.remove(local_file_path)


<Response [400]>


KeyError: 'etag'

In [18]:
    with open(local_file_path, 'rb') as file:
        while True:
            data = file.read(part_size)
            if not data:
                break

            # Generate pre-signed URL for the part
            presigned_url = s3_client.generate_presigned_url(
                ClientMethod='upload_part',
                Params={
                    'Bucket': s3_bucket_name,
                    'Key': s3_object_key,
                    'UploadId': upload_id,
                    'PartNumber': part_number,
                }
            )

            # Upload the part
            response = requests.put(presigned_url, data=data)
            print(response)
            etag = response.headers['ETag']

            # Keep track of uploaded part details
            part_info['Parts'].append({
                'PartNumber': part_number,
                'ETag': etag
            })

<Response [400]>


KeyError: 'etag'